In [7]:
import numpy as np
import math

In [74]:
class Scheduler:
    def __init__(self, pareto_a=5.5, pareto_m=500, time_between_clients=11, 
                 initial_clients_count=100, generated_clients_count=500):
        self.pareto_a = pareto_a
        self.pareto_m = pareto_m

        self.time_between_clients = time_between_clients

        A_mean = (pareto_m * pareto_a / (pareto_a - 1))
        A_std = math.sqrt((pareto_a / (pareto_a-2)) * (pareto_m / (pareto_a - 1)) ** 2)
        A = A_mean + A_std

        time_between_clients_mean = time_between_clients
        time_between_clients_std = math.sqrt(time_between_clients)
        λ = 1 / (time_between_clients_mean - time_between_clients_std)
        self.r = (λ * A)

        self.initial_clients_count = initial_clients_count
        self.generated_clients_count = generated_clients_count

        self.processed_clients = []
        self.all_clients = []

        self.queue = []
        self.t = 0
    
    def clean(self):
        self.processed_clients = []
        self.all_clients = []

        self.queue = []
        self.t = 0
    
    def generate_initial_clients(self):
        for i in range(self.initial_clients_count):
            Ak = int((np.random.pareto(self.pareto_a) + 1) * self.pareto_m)
            rk_mean = int(np.random.uniform(self.r / 5.5, self.r * (10 / 5.5)))
            Rk = int(np.random.uniform(0, Ak))
            Tk0 = 0
            client = [Tk0, Ak, Rk, rk_mean]
            self.queue.append(client)
        
    def generate_clients(self):
        T0 = 0
        for i in range(self.generated_clients_count):
            Ak = int((np.random.pareto(self.pareto_a) + 1) * self.pareto_m)
            rk_mean = int(np.random.uniform(self.r / 5.5, self.r * (10 / 5.5)))
            T0 += int(np.random.exponential(self.time_between_clients))
            Rk = 0
            client = [T0, Ak, Rk, rk_mean]
            self.all_clients.append(client)
    
    def index_client(self, index_strategy, rk, Rk, rk_mean, Tk, Tk0):
        if index_strategy == "1":
            return rk
        elif index_strategy == "2":
            return -Tk0
        elif index_strategy == "3":
            return rk * Tk / Rk
        elif index_strategy == "4":
            return rk / Rk
        elif index_strategy == "5":
            return rk / (Tk + 1e-20) 

    def run_simulation(self, strategy="4"):
        history = []
        huawei_quality_metric = 0.
        
        while len(self.all_clients) != 0:
            self.t += 1

            j = 0
            while ((j < len(self.all_clients)) and (self.all_clients[j][0] <= self.t)):
                j += 1
            self.queue += self.all_clients[:j]
            self.all_clients[:j] = []

            max_index = -1e20
            argmax_index = 0
            rk_argmax = 0
            for k in range(len(self.queue)):
                Tk0 = self.queue[k][0]
                Tk = self.t - Tk0
                rk_mean = self.queue[k][3]
                rk = int(np.random.uniform(rk_mean * 0.7, rk_mean * 1.3))
                Rk = self.queue[k][2] + 1e-10
                
                index_k = self.index_client(strategy, rk, Rk, rk_mean, Tk, Tk0)
                if index_k > max_index :
                    max_index = index_k
                    argmax_index = k
                    rk_argmax = rk

            if len(self.queue) > 0:
                self.queue[argmax_index][2] += rk_argmax
                if self.queue[argmax_index][2] >= self.queue[argmax_index][1]:
                    self.queue[argmax_index].append(self.t - self.queue[argmax_index][0])
                    self.processed_clients.append(self.queue[argmax_index])
                    del self.queue[argmax_index]
            history.append(len(self.queue))
            
        for i in range(len(self.processed_clients)):
            huawei_quality_metric += self.processed_clients[i][1] / self.processed_clients[i][-1]
        huawei_quality_metric /= len(self.processed_clients)
        
        return history, huawei_quality_metric

In [81]:
scheduler = Scheduler()
scheduler.generate_initial_clients()
scheduler.generate_clients()
history, metric = scheduler.run_simulation(strategy="5")
scheduler.clean()
print(metric)
scheduler.generate_initial_clients()
scheduler.generate_clients()
history, metric = scheduler.run_simulation(strategy="5")
scheduler.clean()
print(metric)

69.43873239391831
70.3989540984678
